In [1]:
import os    
%pylab inline
import numpy as np
import pandas as pd
from scipy.misc import imread
from sklearn.metrics import accuracy_score

#os.environ['THEANO_FLAGS'] = "device=gpu1"  
#os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu0,floatX=float32,lib.cnmem=1"
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu0,floatX=float32"
import theano

from keras.layers import Input, Dense
from keras.layers import Activation, Dropout, Convolution2D, Flatten, MaxPooling2D, Reshape, InputLayer
from keras.models import Sequential, Model
from keras.optimizers import *
from keras.utils.np_utils import *
from keras.constraints import maxnorm


import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cross_validation import train_test_split

Populating the interactive namespace from numpy and matplotlib
Couldn't import dot_parser, loading of dot files will not be possible.


Using gpu device 0: GeForce 920M (CNMeM is disabled, cuDNN not available)
Using Theano backend.


In [2]:
X_train=np.load('/media/mrafi123/UStore/Dimensionality-Reduction/data/CIFAR10/X_train.npy')
X_test=np.load('/media/mrafi123/UStore/Dimensionality-Reduction/data/CIFAR10/X_test.npy')
y_train=np.load('/media/mrafi123/UStore/Dimensionality-Reduction/data/CIFAR10/y_train.npy')
y_test=np.load('/media/mrafi123/UStore/Dimensionality-Reduction/data/CIFAR10/y_test.npy')

In [3]:
print (X_train.shape)
print (y_train.shape)
print (X_test.shape)
print (y_test.shape)

(50000, 32, 32, 3)
(50000,)
(10000, 32, 32, 3)
(10000,)


In [4]:
#x_train = X_train.reshape((len(X_train), np.prod(X_train.shape[1:])))
#x_test = X_test.reshape((len(X_test), np.prod(X_test.shape[1:])))

x_train=X_train
x_test = X_test
print (x_train.shape)
print (x_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [5]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

In [6]:
# Normalize the data: subtract the mean image
mean_image = np.mean(x_train, axis=0)
x_train -= mean_image
x_test -= mean_image

In [15]:
x_train /= np.std(x_train, axis = 0) # normalize
x_test /= np.std(x_test, axis = 0) # normalize

In [6]:
s_train, s_valid = train_test_split(x_train, test_size=0.2)
label_train, label_valid = train_test_split(y_train, test_size=0.2)


In [7]:
print (s_train.shape)
print (s_valid.shape)

print (label_train.shape)
print (label_valid.shape)
print (x_test.shape)
print (y_test.shape)
label_train=to_categorical(label_train)
label_valid=to_categorical(label_valid)
label_test= to_categorical(y_test)
print (label_train.shape)
print (label_valid.shape)
print (label_test.shape)



(40000, 32, 32, 3)
(10000, 32, 32, 3)
(40000,)
(10000,)
(10000, 32, 32, 3)
(10000,)
(40000, 10)
(10000, 10)
(10000, 10)


In [8]:
# To stop potential randomness
seed = 128
rng = np.random.RandomState(seed)

In [96]:
# define vars
input_num_units = 3072
#hidden_num_units = 50
#hidden_num_units = 500


hidden1_num_units = 500
hidden2_num_units = 500
hidden3_num_units = 500
hidden4_num_units = 500
hidden5_num_units = 500

output_num_units = 10

epochs = 25
batch_size = 128

In [63]:
# Shallow Network
model = Sequential([
  Dense(output_dim=hidden_num_units, input_dim=input_num_units, activation='relu'),
  Dense(output_dim=output_num_units, input_dim=hidden_num_units, activation='softmax'),
])

In [83]:
# Deep network
model = Sequential([
 Dense(output_dim=hidden1_num_units, input_dim=input_num_units, activation='relu'),
 Dense(output_dim=hidden2_num_units, input_dim=hidden1_num_units, activation='relu'),
 Dense(output_dim=hidden3_num_units, input_dim=hidden2_num_units, activation='relu'),
 Dense(output_dim=hidden4_num_units, input_dim=hidden3_num_units, activation='relu'),
 Dense(output_dim=hidden5_num_units, input_dim=hidden4_num_units, activation='relu'),

Dense(output_dim=output_num_units, input_dim=hidden5_num_units, activation='softmax'),
 ])

In [97]:
dropout_ratio = 0.2

model = Sequential([
 Dense(output_dim=hidden1_num_units, input_dim=input_num_units, activation='relu'),
 Dropout(dropout_ratio),
 Dense(output_dim=hidden2_num_units, input_dim=hidden1_num_units, activation='relu'),
 Dropout(dropout_ratio),
 Dense(output_dim=hidden3_num_units, input_dim=hidden2_num_units, activation='relu'),
 Dropout(dropout_ratio),
 Dense(output_dim=hidden4_num_units, input_dim=hidden3_num_units, activation='relu'),
 Dropout(dropout_ratio),
 Dense(output_dim=hidden5_num_units, input_dim=hidden4_num_units, activation='relu'),
 Dropout(dropout_ratio),

Dense(output_dim=output_num_units, input_dim=hidden5_num_units, activation='softmax'),
 ])

In [98]:
# compile the model with necessary attributes
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [99]:
trained_model = model.fit(
    s_train, label_train, nb_epoch=epochs, batch_size=batch_size, 
    validation_data=(s_valid, label_valid))

Train on 40000 samples, validate on 10000 samples
Epoch 1/25
40000/40000 [==============================] - 11s - loss: 2.3277 - acc: 0.0996 - val_loss: 2.3046 - val_acc: 0.0966
Epoch 2/25
40000/40000 [==============================] - 11s - loss: 2.3053 - acc: 0.0984 - val_loss: 2.3034 - val_acc: 0.1009
Epoch 3/25
40000/40000 [==============================] - 11s - loss: 2.3041 - acc: 0.0980 - val_loss: 2.3031 - val_acc: 0.1009
Epoch 4/25
40000/40000 [==============================] - 11s - loss: 2.3034 - acc: 0.0995 - val_loss: 2.3029 - val_acc: 0.0961
Epoch 5/25
40000/40000 [==============================] - 11s - loss: 2.3028 - acc: 0.0977 - val_loss: 2.3113 - val_acc: 0.1019
Epoch 6/25
40000/40000 [==============================] - 11s - loss: 2.3034 - acc: 0.1014 - val_loss: 2.3031 - val_acc: 0.0942
Epoch 7/25
40000/40000 [==============================] - 11s - loss: 2.3034 - acc: 0.0998 - val_loss: 2.3031 - val_acc: 0.0969
Epoch 8/25
40000/40000 [==============================

In [9]:
# reshape data
train_x_temp = s_train.reshape(-1,3,32,32)
val_x_temp = s_valid.reshape(-1,3,32,32)
test_x_temp=x_test.reshape(-1,3,32,32)

#train_x_temp = s_train
#val_x_temp = s_valid
#test_x_temp=x_test

print(train_x_temp.shape)
print(val_x_temp.shape)
print(test_x_temp.shape)

dropout_ratio = 0.2


# define vars
input_shape = (3072,)
input_reshape = (3, 32, 32)

conv_num_filters = 5
conv_filter_size = 5

pool_size = (2, 2)

hidden_num_units = 512
output_num_units = 10

epochs = 25
batch_size = 32

model = Sequential([
 #InputLayer(input_shape=input_reshape),

 Convolution2D(32, 3, 3, border_mode='same',activation='relu',W_constraint=maxnorm(3),input_shape=input_reshape),
 #MaxPooling2D(pool_size=pool_size),
 Dropout(dropout_ratio),
 Convolution2D(32, 3, 3, border_mode='same',activation='relu',W_constraint=maxnorm(3)),
 MaxPooling2D(pool_size=pool_size),
 Flatten(),
 Dense(output_dim=hidden_num_units, activation='relu',W_constraint=maxnorm(3)),
 Dropout(0.5),
        
 Dense(output_dim=output_num_units, input_dim=hidden_num_units, activation='softmax'),
])

#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Compile model
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

trained_model_conv = model.fit(train_x_temp, label_train, 
                               nb_epoch=epochs, batch_size=batch_size, 
                               validation_data=(val_x_temp, label_valid))

(40000, 3, 32, 32)
(10000, 3, 32, 32)
(10000, 3, 32, 32)
Train on 40000 samples, validate on 10000 samples
Epoch 1/25
40000/40000 [==============================] - 99s - loss: 2.3042 - acc: 0.1002 - val_loss: 2.3030 - val_acc: 0.0989
Epoch 2/25
40000/40000 [==============================] - 96s - loss: 2.3033 - acc: 0.0995 - val_loss: 2.3037 - val_acc: 0.0975
Epoch 3/25
40000/40000 [==============================] - 98s - loss: 2.3032 - acc: 0.0955 - val_loss: 2.3026 - val_acc: 0.0993
Epoch 4/25
40000/40000 [==============================] - 99s - loss: 2.3028 - acc: 0.1012 - val_loss: 2.3034 - val_acc: 0.0975
Epoch 5/25
40000/40000 [==============================] - 98s - loss: 2.3029 - acc: 0.0990 - val_loss: 2.3028 - val_acc: 0.1000
Epoch 6/25
40000/40000 [==============================] - 101s - loss: 2.3025 - acc: 0.1029 - val_loss: 2.3028 - val_acc: 0.0999
Epoch 7/25
40000/40000 [==============================] - 98s - loss: 2.3021 - acc: 0.1035 - val_loss: 2.3027 - val_acc: 0.0

In [10]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 32, 32, 32)    896         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 32, 32, 32)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 32, 32, 32)    9248        dropout_1[0][0]                  
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 32, 16, 16)    0           convolution2d_2[0][0]            
___________________________________________________________________________________________

In [86]:
#pred = model.predict_classes(x_test)
pred = model.predict_classes(x_test)

 9984/10000 [============================>.] - ETA: 0s

In [11]:
#pred = model.predict_classes(x_test)
pred = model.predict_classes(test_x_temp)

10000/10000 [==============================] - 6s     


In [12]:
pred

array([7, 6, 1, ..., 4, 2, 1])

In [13]:
pred= to_categorical(pred)

In [14]:
pred

array([[ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.]])

In [15]:
label_test

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  1.,  0.,  0.]])

In [16]:
num=len(label_test)
r=0
w=0
for i in range(num):
        #print ('y_pred ',y_pred[i])
        #print ('labels ',labels[i])
        #without the use of all() returns error truth value of an array with more than one element is ambiguous
        #if y_pred[i].all() == labels[i].all():
        if np.array_equal(pred[i],label_test[i]):
            r+=1
        else:
            w+=1
print ("tested ",  num, "digits")
print ("correct: ", r, "wrong: ", w, "error rate: ", float(w)*100/(r+w), "%")
print ("got correctly ", float(r)*100/(r+w), "%")

tested  10000 digits
correct:  928 wrong:  9072 error rate:  90.72 %
got correctly  9.28 %
